In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.stattools import durbin_watson
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score

import warnings
warnings.filterwarnings('ignore') 

plt.style.use('ggplot') 
sns.set_theme(style="whitegrid") 



# Import

In [3]:
pd.read_excel("2022_코스피.xlsx").to_csv("KOSPI_2022.csv", index=False, encoding='utf-8-sig')
pd.read_excel("2023_코스피.xlsx").to_csv("KOSPI_2023.csv", index=False, encoding='utf-8-sig')
pd.read_excel("2024_코스피.xlsx").to_csv("KOSPI_2024.csv", index=False, encoding='utf-8-sig')

pd.read_excel("2022_코스닥.xlsx").to_csv("KOSDAQ_2022.csv", index=False, encoding='utf-8-sig')
pd.read_excel("2023_코스닥.xlsx").to_csv("KOSDAQ_2023.csv", index=False, encoding='utf-8-sig')
pd.read_excel("2024_코스닥.xlsx").to_csv("KOSDAQ_2024.csv", index=False, encoding='utf-8-sig')

pd.read_excel("2022_시장.xlsx").to_csv("MARKET_2022.csv", index=False, encoding='utf-8-sig')
pd.read_excel("2023_시장.xlsx").to_csv("MARKET_2023.csv", index=False, encoding='utf-8-sig')
pd.read_excel("2024_시장.xlsx").to_csv("MARKET_2024.csv", index=False, encoding='utf-8-sig')

pd.read_excel("2022_한국.xlsx").to_csv("KOREA_2022.csv", index=False, encoding='utf-8-sig')
pd.read_excel("2023_한국.xlsx").to_csv("KOREA_2023.csv", index=False, encoding='utf-8-sig')
pd.read_excel("2024_한국.xlsx").to_csv("KOREA_2024.csv", index=False, encoding='utf-8-sig')


In [4]:
KOSPI_2022 = pd.read_csv('KOSPI_2022.csv')
KOSPI_2023 = pd.read_csv('KOSPI_2023.csv')
KOSPI_2024 = pd.read_csv('KOSPI_2024.csv')

KOSDAQ_2022 = pd.read_csv('KOSDAQ_2022.csv')
KOSDAQ_2023 = pd.read_csv('KOSDAQ_2023.csv')
KOSDAQ_2024 = pd.read_csv('KOSDAQ_2024.csv')

MARKET_2022 = pd.read_csv('MARKET_2022.csv')
MARKET_2023 = pd.read_csv('MARKET_2023.csv')
MARKET_2024 = pd.read_csv('MARKET_2024.csv')

KOREA_2022 = pd.read_csv('KOREA_2022.csv')
KOREA_2023 = pd.read_csv('KOREA_2023.csv')
KOREA_2024 = pd.read_csv('KOREA_2024.csv')

In [5]:
KOREA_2022

,제목,내용,댓글,작성일자,URL
0,요즘 한국 가계부채의 특징,1. 원금 상환 없이 이자만 냄2. 가능한 최대한도로 대출 규모 크게 끌어옴3. 2...,퀀트5년차 (2022.12.31 22:12:35): 집주인이 빚 내서 산 아파트에 ...,2022-12-31 22:11:16,https://gall.dcinside.com/mgallery/board/view/...
1,지금 전문가들이 IMF 급으로 예상하는 이유가,2년물 5년물 10년물 할 것 없이근래 미 국채 금리가 쩜상으로 뛰었음차트에서 봐도...,"ㅇㅇ (2022.12.31 03:11:30): 내년에 좆스피 1450, 부동산 1/...",2022-12-31 01:45:13,https://gall.dcinside.com/mgallery/board/view/...
2,중국 비밀경찰서 식당 주인 근황,"밖에서 기자들 데리고 긴급 기자회견을 때림대충 내용은...""나는 평범한 식당 주인이...",ㅇㅇ (2022.12.29 16:43:04): 짱깨가 저렇게 당당하게 설치게 놔둔게...,2022-12-29 16:40:59,https://gall.dcinside.com/mgallery/board/view/...
3,한국증시 시발진짜 존나 떨어지네,한국증시 시발진짜 존나 떨어지네 - dc official App,ㅇㅇ (2022.12.29 14:21:07): 유에스 아미!!!!!!!!!!!!!!...,2022-12-29 14:20:16,https://gall.dcinside.com/mgallery/board/view/...
4,와 ㅆㅂ 한국장만 ㅈㄴ빼네,너무한다,ㅅㅅㅅ (2022.12.29 14:18:52): 코스피가 넘 많이 올랐음|ㅇㅇ (2...,2022-12-29 14:18:02,https://gall.dcinside.com/mgallery/board/view/...
...,...,...,...,...,...
638,한국 기자들이 배당을 보는 시각,1 이미지 순서 ON2 이미지 순서 ON3 이미지 순서 ON공산주의+국수주의+반자본주의,ㅇㅇ (2022.01.05 21:46:37): 한전도 돈 많이 벌면 대표이사 국감 ...,2022-01-05 21:35:35,https://gall.dcinside.com/mgallery/board/view/...
639,은마전설,"Long Long time after in Gaxaxy, 우주력 13250년----...",ㅇㅇ (2022.01.04 14:06:35): 2020년이 아닌 2002년에도 재건...,2022-01-04 14:03:58,https://gall.dcinside.com/mgallery/board/view/...
640,"""중국인 한국에 영구 정착길 열린다""",ㅎㅎ,ㅇㅇ (2022.01.04 12:50:45): 지랄 병신들 국내인재는 잡을생각이 없...,2022-01-04 12:40:07,https://gall.dcinside.com/mgallery/board/view/...
641,1880억 횡령해서 주식 샀다가 -7% 손절(ㅣ보),한국 쥬식시장 코미디네 씨벌 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,ㅇㅇ (2022.01.03 11:58:30): 손절이 ㄹㅇ 코미디 ㅋㅋㅋㅋ|ㅇㅇ (...,2022-01-03 11:57:19,https://gall.dcinside.com/mgallery/board/view/...


# 데이터별로 구조(column)가 달라서 각 구조에 맞게 나눈 뒤, 게시글 내용을 월별로 하나로 묶음

In [6]:
# 파일 목록 정의 (구조에 따라 구분)
same_structure_files = [
    "KOSDAQ_2023.csv", "KOSDAQ_2024.csv", "KOSPI_2024.csv", 
    "KOREA_2023.csv", "KOREA_2024.csv", "KOSPI_2022.csv",
    "MARKET_2024.csv", "MARKET_2023.csv", "MARKET_2022.csv"
]

different_structure_files = ["KOSDAQ_2022.csv", "KOSPI_2023.csv", "KOREA_2022.csv"]

# 동일한 구조에 대해 작업 수행
grouped_data = {}

for file_name in same_structure_files:
    # 데이터 로드
    df = pd.read_csv(file_name)
    # '작성일자'를 datetime으로 변환
    df['Column4'] = pd.to_datetime(df['Column4'], errors='coerce')
    # '내용' 열을 문자열로 변환
    df['Column2'] = df['Column2'].astype(str)
    # 월별로 '내용'을 묶는 작업
    df_grouped = df.groupby(df['Column4'].dt.to_period('M'))['Column2'].apply(' '.join).reset_index()
    df_grouped.columns = ['Month', 'Content']
    # 결과 저장
    grouped_data[file_name] = df_grouped

# 다른 구조에 대해 작업 수행
for file_name in different_structure_files:
    # 데이터 로드
    df = pd.read_csv(file_name)
    # '작성일자'를 datetime으로 변환
    df['작성일자'] = pd.to_datetime(df['작성일자'], errors='coerce')
    # '내용' 열을 문자열로 변환
    df['내용'] = df['내용'].astype(str)
    # 월별로 '내용'을 묶는 작업
    df_grouped = df.groupby(df['작성일자'].dt.to_period('M'))['내용'].apply(' '.join).reset_index()
    df_grouped.columns = ['Month', 'Content']
    # 결과 저장
    grouped_data[file_name] = df_grouped

# 모든 결과 출력 또는 저장
for file_name, df_grouped in grouped_data.items():
    output_file = f"grouped_{file_name}"
    df_grouped.to_csv(output_file, index=False)
    print(f"{file_name} 작업 완료. 결과는 {output_file}에 저장됨.")


KOSDAQ_2023.csv 작업 완료. 결과는 grouped_KOSDAQ_2023.csv에 저장됨.
KOSDAQ_2024.csv 작업 완료. 결과는 grouped_KOSDAQ_2024.csv에 저장됨.
KOSPI_2024.csv 작업 완료. 결과는 grouped_KOSPI_2024.csv에 저장됨.
KOREA_2023.csv 작업 완료. 결과는 grouped_KOREA_2023.csv에 저장됨.
KOREA_2024.csv 작업 완료. 결과는 grouped_KOREA_2024.csv에 저장됨.
KOSPI_2022.csv 작업 완료. 결과는 grouped_KOSPI_2022.csv에 저장됨.
MARKET_2024.csv 작업 완료. 결과는 grouped_MARKET_2024.csv에 저장됨.
MARKET_2023.csv 작업 완료. 결과는 grouped_MARKET_2023.csv에 저장됨.
MARKET_2022.csv 작업 완료. 결과는 grouped_MARKET_2022.csv에 저장됨.
KOSDAQ_2022.csv 작업 완료. 결과는 grouped_KOSDAQ_2022.csv에 저장됨.
KOSPI_2023.csv 작업 완료. 결과는 grouped_KOSPI_2023.csv에 저장됨.
KOREA_2022.csv 작업 완료. 결과는 grouped_KOREA_2022.csv에 저장됨.


In [13]:
KOSPI_2022_text = pd.read_csv('/Users/kwaksanga/Documents/y-form 플젝/KOSPI/grouped_KOSPI_2022.csv')
KOSPI_2023_text = pd.read_csv('/Users/kwaksanga/Documents/y-form 플젝/KOSPI/grouped_KOSPI_2023.csv')
KOSPI_2024_text = pd.read_csv('/Users/kwaksanga/Documents/y-form 플젝/KOSPI/grouped_KOSPI_2024.csv')

KOSDAQ_2022_text = pd.read_csv('/Users/kwaksanga/Documents/y-form 플젝/KOSDAQ/grouped_KOSDAQ_2022.csv')
KOSDAQ_2023_text = pd.read_csv('/Users/kwaksanga/Documents/y-form 플젝/KOSDAQ/grouped_KOSDAQ_2023.csv')
KOSDAQ_2024_text = pd.read_csv('/Users/kwaksanga/Documents/y-form 플젝/KOSDAQ/grouped_KOSDAQ_2024.csv')

KOREA_2022_text = pd.read_csv('/Users/kwaksanga/Documents/y-form 플젝/KOREA/grouped_KOREA_2022.csv')
KOREA_2023_text = pd.read_csv('/Users/kwaksanga/Documents/y-form 플젝/KOREA/grouped_KOREA_2023.csv')
KOREA_2024_text = pd.read_csv('/Users/kwaksanga/Documents/y-form 플젝/KOREA/grouped_KOREA_2024.csv')

MARKET_2022_text = pd.read_csv('/Users/kwaksanga/Documents/y-form 플젝/MARKET/grouped_MARKET_2022.csv')
MARKET_2023_text = pd.read_csv('/Users/kwaksanga/Documents/y-form 플젝/MARKET/grouped_MARKET_2023.csv')
MARKET_2024_text = pd.read_csv('/Users/kwaksanga/Documents/y-form 플젝/MARKET/grouped_MARKET_2024.csv')

# 각 키워드별/연도별/게시글 내용 데이터 확보

# 4개의 키워드가 담긴 게시글 내용을 동일 연도끼리 병합

In [14]:
# 데이터 통합 함수 정의
def combine_text_by_month(*dfs):
    combined_df = pd.concat(dfs)
    return combined_df.groupby('Month')['Content'].apply(' '.join).reset_index()

# 2022년 데이터 통합
text_2022_combined = combine_text_by_month(KOSPI_2022_text, KOSDAQ_2022_text, KOREA_2022_text, MARKET_2022_text)
text_2022_combined.to_csv('2022_text_combined.csv', index=False)

# 2023년 데이터 통합
text_2023_combined = combine_text_by_month(KOSPI_2023_text, KOSDAQ_2023_text, KOREA_2023_text, MARKET_2023_text)
text_2023_combined.to_csv('2023_text_combined.csv', index=False)

# 2024년 데이터 통합
text_2024_combined = combine_text_by_month(KOSPI_2024_text, KOSDAQ_2024_text, KOREA_2024_text, MARKET_2024_text)
text_2024_combined.to_csv('2024_text_combined.csv', index=False)

# 결과 미리보기
text_2022_combined.head(), text_2023_combined.head(), text_2024_combined.head()


(     Month                                            Content
 0  2022-01  0128(금) 코스피 지수 예상 : 하락상승 79하락 173오늘 코스피 지수는 48...
 1  2022-02  0228(월) 코스피 지수 예상 : 상승상승 129하락 117오늘 코스피 지수는 2...
 2  2022-03  0331(목) 코스피 지수 예상 : 상승상승 51하락 34오늘 코스피 지수는 10....
 3  2022-04  코스피갤러리 공식 투자 입문서와 함께합니다↘실시간 확인은 여기서https://doc...
 4  2022-05  0531(화) 코스피 지수 예상 : 상승상승 71하락 29오늘 코스피 지수는 16....,
      Month                                            Content
 0  2023-01  0131(화) 코스피 지수 예상 : 하락상승 24하락 50오늘 코스피 지수는 25....
 1  2023-02  0228(화) 코스피 지수 예상 : 강보합상승 25하락 22오늘 코스피 지수는 10...
 2  2023-03  https://m.dcinside.com/board/kospi/2704186?hea...
 3  2023-04  발포제 한길만파던 금양이 배터리 사업한다고 하지만22년 사업보고서 속 금양을 보면 ...
 4  2023-05  여기 새끼들 다 이미 사회에서 죽은 놈들이다   - dc official App 가...,
      Month                                            Content
 0  2024-01  해당 갤러리의 관리자 일동에게 조속히 빠른 사죄를 요구합니다! 평소에는 10원짜리 ...
 1  2024-02  2월은 코갤러들 선택처럼 콜이 승리 1 이미지 순서 ON어제 아침 한국 증시 밸류업...
 2  2024-03  안보여서 뒤진줄 알았더니  코스피갤에 있었노ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ  5년전에

In [15]:
text_2023_combined

,Month,Content
0,2023-01,0131(화) 코스피 지수 예상 : 하락상승 24하락 50오늘 코스피 지수는 25....
1,2023-02,0228(화) 코스피 지수 예상 : 강보합상승 25하락 22오늘 코스피 지수는 10...
2,2023-03,https://m.dcinside.com/board/kospi/2704186?hea...
3,2023-04,발포제 한길만파던 금양이 배터리 사업한다고 하지만22년 사업보고서 속 금양을 보면 ...
4,2023-05,여기 새끼들 다 이미 사회에서 죽은 놈들이다 - dc official App 가...
5,2023-06,"마켓 스파이06.30 16:37 기준 기준1. 모닝리포트[시황요약]-DOW 34,1..."
6,2023-07,"1. 모닝리포트[시황요약]-DOW 35,459.29p (+0.50%) -NASDAQ..."
7,2023-08,nan 평소에 자주 가던 단골가게였는데여알바가 대뜸 뭐 보고 있냐고 묻더라근데 난 ...
8,2023-09,9월은 코갤러들 선택처럼 풋이 승리 코반꿀이다 ㅇㅇ진짜 진지하게 이거 맞음이새끼들 ...
9,2023-10,1 이미지 순서 ON2 이미지 순서 ON코스피 차트3 이미지 순서 ON증권사 전망ㅋ...


In [16]:
text_2024_combined

,Month,Content
0,2024-01,해당 갤러리의 관리자 일동에게 조속히 빠른 사죄를 요구합니다! 평소에는 10원짜리 ...
1,2024-02,2월은 코갤러들 선택처럼 콜이 승리 1 이미지 순서 ON어제 아침 한국 증시 밸류업...
2,2024-03,안보여서 뒤진줄 알았더니 코스피갤에 있었노ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 5년전에도 7억이더...
3,2024-04,1 이미지 순서 ONㄷㄷㄷ 쎄다세좌빨슈카 개새끼 입갤코스피갤러리 하는거 맞네ㅋㅋht...
4,2024-05,- dc official App 1 이미지 순서 ON2 이미지 순서 ON대재인 그립...
5,2024-06,언젠가부터 컄ㅋㅋㅋㅋ 이딴 말투쓰고있고Boji라는 단어가 입에붙어버린다미주갤만보다 ...
6,2024-07,코갤 히스토리 채널(졸업자 퇴출자 등등)CuteSpy: 인버스 애호가. 퇴학김바보:...
7,2024-08,ㄹㅇ좆소기업 다니는 탈모 비만 노총각 아재 느낌- dc official App 현실...
8,2024-09,경험상 내릴자리에서 삼전 버티고 나머지 빠지는 장 나오고 미장 폭락하면그 담날 개씹...
9,2024-10,nan 재명세만아니었으면코스피줫망으로 탄핵하자면서 여론부추겼을텐데못해서 ㅋㅋ 1 이...


In [17]:
text_2022_combined.to_csv('text_2022.csv', index=False)
text_2023_combined.to_csv('text_2023.csv', index=False)
text_2024_combined.to_csv('text_2024.csv', index=False)

# 데이터 준비 완료
# 감성분석 시작

In [18]:
text_2022 = pd.read_csv('text_2022.csv')
text_2023 = pd.read_csv('text_2023.csv')
text_2024 = pd.read_csv('text_2024.csv')

In [19]:
text_2022

,Month,Content
0,2022-01,0128(금) 코스피 지수 예상 : 하락상승 79하락 173오늘 코스피 지수는 48...
1,2022-02,0228(월) 코스피 지수 예상 : 상승상승 129하락 117오늘 코스피 지수는 2...
2,2022-03,0331(목) 코스피 지수 예상 : 상승상승 51하락 34오늘 코스피 지수는 10....
3,2022-04,코스피갤러리 공식 투자 입문서와 함께합니다↘실시간 확인은 여기서https://doc...
4,2022-05,0531(화) 코스피 지수 예상 : 상승상승 71하락 29오늘 코스피 지수는 16....
5,2022-06,0630(목) 코스피 지수 예상 : 하락상승 57하락 65오늘 코스피 지수는 45....
6,2022-07,0729(금) 코스피 지수 예상 : 강보합상승 38하락 37오늘 코스피 지수는 16...
7,2022-08,0831(수) 코스피 지수 예상 : 약보합상승 28하락 30오늘 코스피 지수는 21...
8,2022-09,"여러분이 20, 21년에 늘 외쳤던 그 문장 '아 씨발 좆같다 삼전 다시 코로나 이..."
9,2022-10,1031(월) 코스피 지수 예상 : 상승상승 70하락 48오늘 코스피 지수는 25....


In [20]:
text_2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Month    12 non-null     object
 1   Content  12 non-null     object
dtypes: object(2)
memory usage: 324.0+ bytes


In [21]:
from transformers import BertTokenizer, BertForSequenceClassification

# Hugging Face의 monologg/kobert 모델 로드
kobert_tokenizer = BertTokenizer.from_pretrained("monologg/kobert")
kobert_model = BertForSequenceClassification.from_pretrained("monologg/kobert", num_labels=3)

tokenizer_config.json:   0%|          | 0.00/263 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import numpy as np
import random

# 랜덤 시드 고정
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# BERT 모델 및 토크나이저 로드 (KoBERT 사용 예시)
kobert_tokenizer = BertTokenizer.from_pretrained("monologg/kobert")
kobert_model = BertForSequenceClassification.from_pretrained("monologg/kobert", num_labels=3)

# GPU 사용 설정 (필요시)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
kobert_model.to(device)

# 모델 추론 모드 설정
kobert_model.eval()

# 감성 분석 함수 정의
def calculate_sentiment(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    inputs = {key: val.to(device) for key, val in inputs.items()}  # GPU로 데이터 이동
    with torch.no_grad():  # 그래디언트 계산 비활성화
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    sentiment_score = probs[0][2].item() - probs[0][0].item()  # 긍정 점수 - 부정 점수
    return sentiment_score

# 월별 감성 스코어 계산
def process_monthly_sentiment(df, tokenizer, model):
    df['Sentiment_Score'] = df['Content'].apply(lambda x: calculate_sentiment(x, tokenizer, model))
    return df

# 데이터 처리
text_2022 = process_monthly_sentiment(text_2022, kobert_tokenizer, kobert_model)
text_2023 = process_monthly_sentiment(text_2023, kobert_tokenizer, kobert_model)
text_2024 = process_monthly_sentiment(text_2024, kobert_tokenizer, kobert_model)

# 결과 저장
text_2022.to_csv('text_2022_with_sentiment_BERT.csv', index=False)
text_2023.to_csv('text_2023_with_sentiment_BERT.csv', index=False)
text_2024.to_csv('text_2024_with_sentiment_BERT.csv', index=False)

# 월별 감성 스코어 평균 출력
print(text_2022.groupby('Month')['Sentiment_Score'].mean())
print(text_2023.groupby('Month')['Sentiment_Score'].mean())
print(text_2024.groupby('Month')['Sentiment_Score'].mean())


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Month
2022-01    0.159991
2022-02    0.131458
2022-03    0.113168
2022-04    0.118012
2022-05    0.109325
2022-06    0.124524
2022-07    0.145079
2022-08    0.148799
2022-09    0.140647
2022-10    0.100307
2022-11    0.116201
2022-12    0.155572
Name: Sentiment_Score, dtype: float64
Month
2023-01    0.121640
2023-02    0.130662
2023-03    0.151877
2023-04    0.087077
2023-05    0.123823
2023-06    0.081350
2023-07    0.062565
2023-08    0.166022
2023-09    0.144473
2023-10    0.161178
2023-11    0.167992
2023-12    0.187645
Name: Sentiment_Score, dtype: float64
Month
2024-01    0.186588
2024-02    0.197919
2024-03    0.147973
2024-04    0.174267
2024-05    0.164427
2024-06    0.158346
2024-07    0.152240
2024-08    0.185143
2024-09    0.180373
2024-10    0.177288
2024-11    0.190038
2024-12    0.129965
Name: Sentiment_Score, dtype: float64


In [23]:


text_2022_with_sentiment_BERT = pd.read_csv('text_2022_with_sentiment_BERT.csv')
text_2023_with_sentiment_BERT = pd.read_csv('text_2023_with_sentiment_BERT.csv')
text_2024_with_sentiment_BERT = pd.read_csv('text_2024_with_sentiment_BERT.csv')


# 각 연도의 평균 감성 점수 계산 및 내림차순 정렬
sentiment_2022_sorted = text_2022_with_sentiment_BERT.groupby('Month')['Sentiment_Score'].mean().sort_values(ascending=False)
sentiment_2023_sorted = text_2023_with_sentiment_BERT.groupby('Month')['Sentiment_Score'].mean().sort_values(ascending=False)
sentiment_2024_sorted = text_2024_with_sentiment_BERT.groupby('Month')['Sentiment_Score'].mean().sort_values(ascending=False)

# 결과 저장
sentiment_2022_sorted.to_csv('sentiment_2022_sorted.csv')
sentiment_2023_sorted.to_csv('sentiment_2023_sorted.csv')
sentiment_2024_sorted.to_csv('sentiment_2024_sorted.csv')

# 결과 출력
print("2022 Sentiment Scores (Sorted):")
print(sentiment_2022_sorted)

print("\n2023 Sentiment Scores (Sorted):")
print(sentiment_2023_sorted)

print("\n2024 Sentiment Scores (Sorted):")
print(sentiment_2024_sorted)


2022 Sentiment Scores (Sorted):
Month
2022-01    0.159991
2022-12    0.155572
2022-08    0.148799
2022-07    0.145079
2022-09    0.140647
2022-02    0.131458
2022-06    0.124524
2022-04    0.118012
2022-11    0.116201
2022-03    0.113168
2022-05    0.109325
2022-10    0.100307
Name: Sentiment_Score, dtype: float64

2023 Sentiment Scores (Sorted):
Month
2023-12    0.187645
2023-11    0.167992
2023-08    0.166022
2023-10    0.161178
2023-03    0.151877
2023-09    0.144473
2023-02    0.130662
2023-05    0.123823
2023-01    0.121640
2023-04    0.087077
2023-06    0.081350
2023-07    0.062565
Name: Sentiment_Score, dtype: float64

2024 Sentiment Scores (Sorted):
Month
2024-02    0.197919
2024-11    0.190038
2024-01    0.186588
2024-08    0.185143
2024-09    0.180373
2024-10    0.177288
2024-04    0.174267
2024-05    0.164427
2024-06    0.158346
2024-07    0.152240
2024-03    0.147973
2024-12    0.129965
Name: Sentiment_Score, dtype: float64
